# Example - hubmapinventory - MIBI

In [ ]:
import pandas as pd
from shutil import copytree
from shutil import rmtree
import hubmapbags
import pathlib
import json
import shutil
import yaml
import hubmapinventory

from pathlib import Path
import pandas as pd
from tqdm import tqdm
from warnings import warn as warning
from datetime import datetime
from time import sleep
from pandarallel import pandarallel

instance = 'prod'
token = ''
ncores = 2
pandarallel.initialize(progress_bar=True, nb_workers=ncores)

In [ ]:
# just to list them all, not really needed
assay_types = hubmapbags.apis.get_assay_types( token=token )
print(assay_types)

In [ ]:
!find . -type f -name "*.computing" -exec rm -f {} \;

In [ ]:
assay_types = ['MIBI']

compute_uuids = True
for assay_type in assay_types:
    hubmapbags.utilities.pprint(assay_type)
    print("Retrieving dataset IDs. This might take a while. Be patient.")
    datasets = hubmapbags.apis.get_hubmap_ids(assay_type, token=token)

    for dataset in datasets:
        if (
            dataset["status"] == "Published"
            and dataset["is_primary"]
        ):
            df = hubmapinventory.inventory.create(
                dataset["hubmap_id"],
                token=token,
                ncores=ncores,
                compute_uuids=compute_uuids,
                recompute_file_extension=True,
            )
        else:
            print(f'Avoiding computation of dataset {dataset["hubmap_id"]}({dataset["status"]}).')